In [1]:
using CSV, DataFrames, Dates
using FileIO
using Geodesy

In [2]:
romania_full = CSV.read("../data/romania.csv", DataFrame);
romania = romania_full[romania_full.Datetime .> DateTime(1976,1,1,0,0,0),:]

Row,Datetime,Latitude,Longitude,Depth,Magnitude
,DateTime,Float64,Float64,Float64,Float64
1,1976-02-03T13:29:16.400,45.32,25.99,184.3,2.6
2,1976-08-19T19:03:00.800,45.54,26.37,162.0,2.3
3,1976-09-07T17:38:08.200,45.62,26.5,155.3,3.6
4,1976-10-01T17:50:43.200,45.68,26.49,146.0,6.0
5,1976-10-15T10:54:48,47.1,25.6,17.0,3.9
6,1977-03-04T19:21:54.100,45.77,26.76,94.0,7.4
7,1977-03-04T21:21:01.100,45.22,26.65,141.0,3.0
8,1977-03-04T21:28:07.400,45.23,26.87,22.0,1.5
9,1977-03-04T23:17:14,45.0,27.2,33.0,2.5


In [3]:
vrancea = romania[(romania.Latitude .>= 45.20) .& (romania.Latitude .<= 46.0) .& 
                    (romania.Longitude .>= 26.0) .& (romania.Longitude .<= 27.0) .&
                    (romania.Depth .>= 50.0) .& (romania.Depth .<= 200.0), :]

Row,Datetime,Latitude,Longitude,Depth,Magnitude
,DateTime,Float64,Float64,Float64,Float64
1,1976-08-19T19:03:00.800,45.54,26.37,162.0,2.3
2,1976-09-07T17:38:08.200,45.62,26.5,155.3,3.6
3,1976-10-01T17:50:43.200,45.68,26.49,146.0,6.0
4,1977-03-04T19:21:54.100,45.77,26.76,94.0,7.4
5,1977-03-04T21:21:01.100,45.22,26.65,141.0,3.0
6,1977-03-05T02:35:22,45.63,26.19,121.3,3.0
7,1977-03-05T12:08:42,45.37,26.3,124.0,3.5
8,1977-03-06T13:27:06.300,45.83,26.63,112.3,2.6
9,1977-03-08T18:44:56.500,45.96,26.44,80.4,1.9


In [4]:
region = "vrancea"
side = 5

5

In [5]:
# mapcoords= Dict("minLat"=>minimum(vrancea.Latitude),
#             "maxLat"=>maximum(vrancea.Latitude),
#             "minLon"=>minimum(vrancea.Longitude),
#             "maxLon"=>maximum(vrancea.Longitude),
#             "minDepth"=>minimum(vrancea.Depth),
#             "maxDepth"=>maximum(vrancea.Depth),
#             "minMag"=>minimum(vrancea.Magnitude),
#             "maxMag"=>maximum(vrancea.Magnitude)
# )

In [6]:
mapcoords

UndefVarError: UndefVarError: mapcoords not defined

In [7]:
minLat=minimum(vrancea.Latitude)
maxLat=maximum(vrancea.Latitude)
minLon=minimum(vrancea.Longitude)
maxLon=maximum(vrancea.Longitude)
minDepth=minimum(vrancea.Depth)
maxDepth=maximum(vrancea.Depth);

In [8]:
# if region=="vrancea" || region=="romania"
#     longitudeInKm = 79
# end;

In [9]:
# Number of cubes needed for each dimension
# Use ceil() to round up to integer (in order to encompass whole region)
# (some dimensions will not be perfectly divisible with the side chosen for cubes)

In [10]:
x0_lla = LLA(minLat,minLon,-minDepth)
xf_lla = LLA(maxLat,minLon,-minDepth)
lat_dist_in_km = Geodesy.euclidean_distance(xf_lla,x0_lla) / 1000 # eucl_dist returns in meters
# xreal, without ceiling, used when assigning earthquakes to certain cubes
xreal = lat_dist_in_km / 5

# Actual number of cubes (integer) on Latitude distance to cover whole volume
x = ceil(Int, lat_dist_in_km / 5)


println("real value of split: ",xreal,"\nint number of cubes on Latitude dim: ",x)

real value of split: 17.782676350109007
int number of cubes on Latitude dim: 18


In [11]:
y0_lla = LLA(minLat,minLon,-minDepth)
yf_lla = LLA(minLat,maxLon,-minDepth)
lon_dist_in_km = Geodesy.euclidean_distance(yf_lla,y0_lla) / 1000

# yreal, without ceiling, used when assigning earthquakes to certain cubes
yreal = lon_dist_in_km / 5

# Actual number of cubes (integer) on Longitude distance to cover whole volume
y = ceil(Int, lon_dist_in_km / 5)

println("real value of split: ",yreal,"\nint number of cubes on Longitude dim: ",y)

real value of split: 15.714087371866526
int number of cubes on Longitude dim: 16


In [12]:
# Depth is already in km 
# Number of cubes on depth distance to cover whole volume

# zreal, without ceiling, used when assigning earthquakes to certain cubes
zreal = (maxDepth-minDepth) / side
# Actual number of cubes (integer) on Depth distance to cover whole volume
z = ceil(Int, (maxDepth-minDepth) / side)

println("real value of split: ",zreal,"\nint number of cubes on Depth dim: ",z)

real value of split: 29.2
int number of cubes on Depth dim: 30


In [13]:
# How many cubes in this region
number_of_cubes = x * y * z

# The most numerous dimension (needed for later when calculating cubeIndex)
maxDim = maximum((x,y,z))

println("Number of cubes: ", number_of_cubes,"\nLongest dimension: ",maxDim)

Number of cubes: 8640
Longest dimension: 30


In [14]:
# xLatitude = [floor(Int,((i-minLat)*x/(maxLat-minLat))+1) for i in vrancea.Latitude]
# Let this mistake in. Using x is wrong because i ceiled the value. 
# This results in mistake when calcualting cube because I use +1 after calculation then floor
# Use xreal / yreal / zreal !

xLatitude = [floor(Int,((i-minLat)*xreal/(maxLat-minLat))+1) for i in vrancea.Latitude];

yLongitude = [floor(Int,((i-minLon)*yreal/(maxLon-minLon))+1) for i in vrancea.Longitude];

zDepth = [floor(Int,((i-minDepth)*zreal/(maxDepth-minDepth))+1) for i in vrancea.Depth];

In [45]:
cubeIndex = String[]
for i in eachindex(xLatitude)
    push!(cubeIndex, string(xLatitude[i]-1+x*(yLongitude[i]-1)+x*y*(zDepth[i]-1)))
end
cubeIndex

7890-element Vector{String}:
 "6433"
 "6183"
 "5608"
 "2514"
 "5364"
 "4077"
 "4107"
 "3632"
 "1852"
 "3442"
 ⋮
 "5303"
 "2207"
 "5319"
 "5069"
 "2189"
 "3319"
 "3571"
 "2190"
 "4780"

In [229]:
# vrancea.xLatitude = xLatitude
# vrancea.yLongitude = yLongitude
# vrancea.zDepth = zDepth
vrancea.cubeIndex = cubeIndex

7890-element Vector{Int64}:
 6433
 6183
 5608
 2514
 5364
 4077
 4107
 3632
 1852
 3442
    ⋮
 5303
 2207
 5319
 5069
 2189
 3319
 3571
 2190
 4780

In [230]:
vrancea

Row,Datetime,Latitude,Longitude,Depth,Magnitude,cubeIndex
,DateTime,Float64,Float64,Float64,Float64,Int64
1,1976-08-19T19:03:00.800,45.54,26.37,162.0,2.3,6433
2,1976-09-07T17:38:08.200,45.62,26.5,155.3,3.6,6183
3,1976-10-01T17:50:43.200,45.68,26.49,146.0,6.0,5608
4,1977-03-04T19:21:54.100,45.77,26.76,94.0,7.4,2514
5,1977-03-04T21:21:01.100,45.22,26.65,141.0,3.0,5364
6,1977-03-05T02:35:22,45.63,26.19,121.3,3.0,4077
7,1977-03-05T12:08:42,45.37,26.3,124.0,3.5,4107
8,1977-03-06T13:27:06.300,45.83,26.63,112.3,2.6,3632
9,1977-03-08T18:44:56.500,45.96,26.44,80.4,1.9,1852


In [232]:
# A half cube value in every direction
x_half_cube = (maxLat-minLat)/(2*xreal)
y_half_cube = (maxLon-minLon)/(2*yreal)
# For z Should always be side/2 (side=5 => 2.5 km)
z_half_cube = (maxDepth-minDepth)/(2*zreal)

2.5

In [234]:
# Formula is: minimum + half value * (2n-1) where n is a cube index in that direction
cubeLatitude=[round(minLat + x_half_cube*(2*i-1), digits = 4) for i in xLatitude]
cubeLongitude=[round(minLon + y_half_cube*(2*n-1), digits = 4) for n in yLongitude]
cubeDepth=[round(minDepth + z_half_cube*(2*n-1), digits = 4) for n in zDepth];

In [235]:
vrancea_cubes = DataFrame(
    # index of the cube
    cubeIndex = cubeIndex,
    # Index on each respective dimension
    xLatitude = xLatitude,
    yLongitude = yLongitude,
    zDepth = zDepth,
    # Position of cube center in every respective dimension
    cubeLatitude = cubeLatitude,
    cubeLongitude = cubeLongitude,
    cubeDepth = cubeDepth
    )

Row,cubeIndex,xLatitude,yLongitude,zDepth,cubeLatitude,cubeLongitude,cubeDepth
,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,6433,8,6,23,45.5374,26.35,162.5
2,6183,10,8,22,45.6274,26.4773,157.5
3,5608,11,8,20,45.6724,26.4773,147.5
4,2514,13,12,9,45.7623,26.7318,92.5
5,5364,1,11,19,45.2225,26.6682,142.5
6,4077,10,3,15,45.6274,26.1591,122.5
7,4107,4,5,15,45.3575,26.2864,122.5
8,3632,15,10,13,45.8523,26.6046,112.5
9,1852,17,7,7,45.9423,26.4136,82.5


In [238]:
unique!(vrancea_cubes)

Row,cubeIndex,xLatitude,yLongitude,zDepth,cubeLatitude,cubeLongitude,cubeDepth
,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,6433,8,6,23,45.5374,26.35,162.5
2,6183,10,8,22,45.6274,26.4773,157.5
3,5608,11,8,20,45.6724,26.4773,147.5
4,2514,13,12,9,45.7623,26.7318,92.5
5,5364,1,11,19,45.2225,26.6682,142.5
6,4077,10,3,15,45.6274,26.1591,122.5
7,4107,4,5,15,45.3575,26.2864,122.5
8,3632,15,10,13,45.8523,26.6046,112.5
9,1852,17,7,7,45.9423,26.4136,82.5


In [240]:
vrancea

Row,Datetime,Latitude,Longitude,Depth,Magnitude,cubeIndex
,DateTime,Float64,Float64,Float64,Float64,Int64
1,1976-08-19T19:03:00.800,45.54,26.37,162.0,2.3,6433
2,1976-09-07T17:38:08.200,45.62,26.5,155.3,3.6,6183
3,1976-10-01T17:50:43.200,45.68,26.49,146.0,6.0,5608
4,1977-03-04T19:21:54.100,45.77,26.76,94.0,7.4,2514
5,1977-03-04T21:21:01.100,45.22,26.65,141.0,3.0,5364
6,1977-03-05T02:35:22,45.63,26.19,121.3,3.0,4077
7,1977-03-05T12:08:42,45.37,26.3,124.0,3.5,4107
8,1977-03-06T13:27:06.300,45.83,26.63,112.3,2.6,3632
9,1977-03-08T18:44:56.500,45.96,26.44,80.4,1.9,1852


In [81]:
energyRelease=[10^(5.24 + 1.44*i) for i in vrancea.Magnitude];
vrancea.energyRelease = energyRelease

7890-element Vector{Float64}:
 3.564511334262439e8
 2.6546055619755363e10
 7.585775750291852e13
 7.870457896951001e15
 3.6307805477010174e9
 3.6307805477010174e9
 1.9054607179632523e10
 9.638290236239705e8
 9.46237161365793e7
 3.6307805477010174e9
 ⋮
 1.342764961137864e9
 5.152286445817561e10
 1.393156802945298e11
 1.342764961137864e9
 1.342764961137864e9
 1.9054607179632523e10
 6.918309709189363e8
 2.6061535499988976e9
 1.8706821403658016e9